# Building and analyzing enhancer-driven TF-gene regulatory networks PANDA and LIONESS

Author: Romana T. Pop<sup>1</sup>, Maud Fagny<sup>2</sup>

<sup>1</sup>Centre for Molecular Medicine Norway (NCMM), University of Oslo, Oslo, Norway

<sup>2</sup>Université Paris-Saclay, INRAE, CNRS, AgroParisTech, GQE – Le Moulon, Gif-sur-Yvette, France

# Introduction
Enhancers are key regulators of gene activity in Eukaryotes, especially during development. Enhancers regulates their target gene by binding transcription factors (TFs) that interact with target gene promoters through 3D chromatin loops<sup>1</sup>. The role of enhancers in wiring the gene regulatory networks is less understood in plants than in animals<sup>2</sup>. Recently, genome-wide studies have allowed to identify many putative active enhancers in plants<sup>3, 4</sup>. The distance between enhancers and their nearest genes was found to vary between species, largely based on genome size<sup>5</sup>. In maize, studies of 3D chromatin data (HiChIP) suggest that 25% of enhancers skip the nearest gene and regulate one further away, and that at least 34% of them regulate multiple genes<sup>4,6</sup>. Finally, it has been shown that genes involved in the same biological pathways tend to be regulated by enhancers carrying binding sites for similar transcription factors<sup>4</sup>, thus creating genome-wide gene regulatory networks that allow the activation of many genes from the same process by using a small number of transcription factors. Identifying enhancers' target genes and deciphering the genome-wide gene regulatory network they articulate is thus an important challenge. It is also still unclear how enhancers and their genome-wide gene regulatory networks arise in plants. It has been proposed that a transposable elements (TEs), which are mobile elements that can duplicate and insert themselves at different loci in the genome, carrying their own regulatory elements, may play a role<sup>7</sup>. 

This tutorial illustrates the analysis that was performed by Fagny et al.<sup>8</sup> on maize data from Oka et al.<sup>7</sup>. Here, we compare the regulatory networks of two types of maize leaves at different developmental stages: dividing young leaves in seedlings (V2-IST) and mature leaves protecting the cob (Husk). We look at differential regulation between the two tissues, identify common and tissue specific regulatory network modules and investigate the role of TEs in wiring stage-specific networks.   

# Necessary data and software

If you are running this locally or adapting it to other data, before beginning the analysis, make sure to install all the necessary software and properly pre-process the data.

This analysis can be run on the server or locally by setting the runserver parameter to 0.

In [ ]:
runserver=1

We also need to set the paths for data in the server.

In [ ]:
if (runserver==1){
    ppath='/opt/data/netZooR/maizeenh/'
} else if (runsevrer==0) {
    ppath=''
}

In addition, some sections of the case study can require some time to run, therefore, we set the precomputed parameter to 1 to load precomputed data. Set to 0 to run the whole tutorial from scratch.

In [ ]:
precomputed=1

In [ ]:
if(runserver==0){
    install.packages("devtools")
    library(devtools)
    install.packages("purr")
    install.packages("tidyr")
    install.packages("data.table")
    install.packages("RColorBrewer")
    install.packages("umap")
    install.packages("ggplot2")
    install.packages("igraph")
    BiocManager::install("IRanges")
    BiocManager::install("limma")
    BiocManager::install("topGO")
    BiocManager::install("Rgraphviz")
    BiocManager::install("GOstats")
    devtools::install_github("netZoo/netZooR", build_vignettes = TRUE)
}

The pipeline requires the following data as input:

* ***lioness_output.Rdata*** : an Rdata file containing the lioness networks for all samples (as a list of data frames);
* ***panda_output.Rdata*** : an Rdata file containing the PANDA output (as a data frame);
* ***samples_annotation.Rdata*** : an Rdata file containing the sample annotation (as a data frame);
* ***gene_expression.Rdata***: an Rdata file containing the gene expression data used to compute the panda networks (as a data frame, optional)
* ***GeneOntologyAnnotation.txt*** : an Rdata file containing the gene ontology annotation (as a txt file);
* ***enhancers_genes.Rdata*** : an Rdata file containing each enhancer-gene candidate pair (as a data frame);
* ***enhancers_TE_overlaps.Rdata*** : an Rdata file containing information about the colocalization of TEs and enhancers (as a data frame);
* ***tfbs_enhancers_TE_genes.Rdata*** : an Rdata file containing information about the colocalization of TEs and TFBS, and the link between enhancers, their predicted TFBS and their potential target genes (as a data frame);
* ***corres_TF_athaliana_zmays.Rdata*** : an Rdata file containing a match table between the names of the transcription factors in *Arabidopsis thaliana* and their orthologs in *Zea mays mays*.

In this tutorial, we will use the enhancer data from Oka et al<sup>7</sup>: a list of enhancers active in either V2-IST or Husk. We integrate these data with gene expression data from the same tissues (6 replicates/tissues). To obtain more reliable gene co-expression data, we also integrate expression data from 11 other tissues from the same maize line obtained in another experiment (AMAIZING GeneAtlas<sup>8</sup>). We will not focus here on generating regulatory networks with PANDA/LIONESS, as this is covered in other tutorials:

[Building PANDA and LIONESS Regulatory Networks from GTEx Gene Expression Data in R](http://netbooks.networkmedicine.org/user/c14c992d-0826-4d80-9f82-c303d37a5167/notebooks/netZooR/ApplicationinGTExData.ipynb)

[Comparing state-specific PANDA networks using pandaR](http://netbooks.networkmedicine.org/user/c14c992d-0826-4d80-9f82-c303d37a5167/notebooks/netZooR/pandaR.ipynb)

[Generating 26 cancer gene regulatory network using TCGA datasets](http://netbooks.networkmedicine.org/user/99920d65-89aa-401b-a223-7dc30bf999f2/notebooks/netZooR/tcga_networks.ipynb)

## A note on priors when inferring enhancers-derived gene regulatory networks

One notable difference between the panda priors used in the tutorials above and the prior used for this analysis is linked to the inference of gene regulatory networks articulated by enhancers instead of promoters. Unlike promoters, enhancers may regulate more than one gene, have a much wider region of action and their nearest gene may not always be their target, particularly in large genomes. Therefore, a slightly different approach was used to create the PANDA prior for this analysis. Enhancers were first annotated with TF binding sites (TFBSs) using FIMO<sup>9</sup>. Because enhancers regulate genes at a distance and can regulate genes located both upstream and downstream, we selected 250kb windows on either side of each enhancer and scanned for genes in these windows. Finally, to account for the fact that enhancers may not always regulate their nearest genes or may regulate multiple genes, we created an edge between each TF that had a predicted binding site in an enhancer and all the genes within +/- 250kb of that enhancer. 

Prior workflow:

* Load expression, TF binding site (TFBS) and enhancer data;
* Extract significant TFBSs predicted in enhancers;
* Select 250kb windows either side of the enhancers;
* Find genes in the +/- 250kb windows on either side of the enhancer (potential target genes);
* Create edges between TF who has a predicted binding site in an enhancer and each potential target gene.

# Load data and libraries

Now that we have installed all the software and prepared the necessary data, we can begin the analysis. First we load the libraries and the data.


In [ ]:
#loading libraries
library(umap)
library(RColorBrewer)
library(limma)
library(topGO)
library(Rgraphviz)
library(netZooR)
library(topGO)
library(data.table)
library(tidyr)
library(purrr)
library(IRanges)
library(ggplot2)
library(igraph)

#loading the panda and lioness networks & GO annotation
load(paste0(ppath, "lioness_output.Rdata"))
load(paste0(ppath, "panda_output.Rdata"))
load(paste0(ppath, "samples_annotation.Rdata"))
load(paste0(ppath, "gene_expression.Rdata"))
GO.file <- paste0(ppath, "GeneOntologyAnnotation.txt")

# Differential targeting analysis between Husk and V2-IST tissues

In this section, we will look at differential targeting between Husk and V2-IST tissues by comparing our PANDA/LIONESS networks. First, we perform a log transformation on the LIONESS network edge weights, to ensure that none of the logtransformed edge weight has a negative or infinite value.

In [ ]:
#function that performs the log transformation
logtrans <- function(x) {
  y <- log(exp(x)+1)
  y[y==Inf] <- x[y==Inf] # replace any infinite values with the original edge values # nolint
  return(y)
}

#performing the transformation on our data
if(precomputed==0){
    ind_net_log <- lapply(lioness, logtrans)
} else {
    load(paste0(ppath, "logtransformed_lioness_results.Rdata"))
}

The PANDA algorithm requires a prior network as input. All potential TF-gene regulatory relationships are integrated in the prior, and edge weights are set to 1. The edge weights are then iteratively updated by PANDA. New edges can also be learnt by integrating information from protein-protein interactions and gene co-expression data. Thus the PANDA and LIONESS networks have more edges than the prior network. Here, we specifically focus on interactions between predicted TFBSs in enhancers and their target genes. Although potential non-canonical or indirect interactions could be revealed through analysis of non-prior edges, we decided to specifically focus on direct interactions, and thus we filter out non-prior edges from the lioness networks. We also filter out any edges that do not take at least 2 different values among the sample-specific networks built from the six replicates of either Husk or V2-IST. 

In [ ]:
if(precomputed==0){
    #filtering out all the additional edges yielded by PANDA/LIONESS and keeping only prior edges # nolint
    ind_net_log_tab <- NULL
    for(i in 1:length(lioness)) {
      ind_net_log_tab <- cbind(ind_net_log_tab, as.numeric(t(as.matrix(ind_net_log[[i]]))[panda$prior == 1])) # filtering non-prior edges in the log-transformed networks # nolint
    }

    #reformatting
    ind_net_log_tab <- data.frame(t(ind_net_log_tab))
    colnames(ind_net_log_tab) <- paste(panda$Gene, panda$TF, sep="_")[panda$prior == 1] # nolint
    rownames(ind_net_log_tab) <- names(ind_net_log)

    #removing edges that do not have different values in at least 2 samples from Husk or V2-IST 
    #necessary for differential targeting analysis
    tissues <- samples.annot[rownames(ind_net_log_tab), "tissue"]
    ind_net_log_tab[,apply(ind_net_log_tab, 2, function(x) length(unique(x))) <= 2] <- NULL # nolint
} else {
    load(paste0(ppath, "logtransformed_lioness_results_table.Rdata"))
}

Although PANDA relies on expression data to generate the networks, PANDA edges and expression data cluster differently from one another. To show this, we plot a UMAP of the gene expression levels and the network edge weights (this step is optional and not necessary for the rest of the analysis). 

In [ ]:
#umap visualisation of edges and expression
custom_config <- umap.defaults
if(nrow(ind_net_log_tab) <= 15) {
  custom_config$n_neighbors <- nrow(ind_net_log_tab) - 1
}
custom_config$n_neighbors <- 6
custom_config$random_state <- 1 # Set the seed
expr_df <- as.data.frame(t(expr)) # expression data
expr_umap <- umap(expr_df[,apply(expr_df, 2, max) <= 2], custom_config) # nolint
ind_net_log_umap <- umap(ind_net_log_tab[,apply(ind_net_log_tab, 2, max) <= 2], custom_config) # nolint

#adding a column for tissue colours to the sample annotation
#tissue_colour <- palette.colors(n = length(unique(samples.annot$tissue)), palette = "Polychrome 36") # nolint
tissue_colour <- c('#5A5156','#E4E1E3','#F6222E','#FE00FA','#16FF32','#3283FE','#FEAF16','#B00068','#1CFFCE','#90AD1C','#2ED9FF','#DEA0FD','#AA0DFE')
names(tissue_colour) <- unique(samples.annot$tissue)
smpl_split <- split(samples.annot, samples.annot$tissue)

for(i in names(smpl_split)) {
  smpl_split[[`i`]]$tissue_colour <- tissue_colour[i]
}

samples.annot <- unsplit(smpl_split, samples.annot$tissue)

#plot umap of gene expression and network edges
layout(matrix(c(1,2), 1, 2, byrow = TRUE), respect = T, heights = c(20,20), widths = c(20,20))
par(mar = c(3, 3, 5, 1) + .1, xpd = TRUE)
plot(expr_umap$layout[, 1], expr_umap$layout[, 2],
     col = samples.annot[rownames(expr_umap$layout), "tissue_colour"],
     pch = ifelse(samples.annot[rownames(expr_umap$layout), "tissue"] %in% c("Husk", "V2-IST"), # nolint
     17, 16), xlab = "", ylab = "", cex.lab = 1, cex.axis = 1)
legend(par("usr")[1]-(par("usr")[2]-par("usr")[1])*1/4, par("usr")[4]+(par("usr")[4]-par("usr")[3])*1/2, 
       legend = names(tissue_colour), col = tissue_colour,
       pch = ifelse(names(tissue_colour) %in% c("Husk", "V2-IST"), 17, 16), ncol = 4, bty = 'n')  # nolint
par(mar = c(3, 3, 5, 1) + .1, xpd = TRUE)
plot(ind_net_log_umap$layout[, 1], ind_net_log_umap$layout[, 2],
     col = samples.annot[rownames(ind_net_log_umap$layout), "tissue_colour"],
     pch = ifelse(samples.annot[rownames(ind_net_log_umap$layout), "tissue"] %in% c("Husk", "V2-IST"), # nolint
     17, 16), xlab = "", ylab = "", cex.lab = 1, cex.axis = 1)
legend(par("usr")[1]-(par("usr")[2]-par("usr")[1])*1/4, par("usr")[4]+(par("usr")[4]-par("usr")[3])*1/2, 
       legend = names(tissue_colour), col = tissue_colour,
       pch = ifelse(names(tissue_colour) %in% c("Husk", "V2-IST"), 17, 16), ncol = 4, bty = 'n')

Next, we use limma to perform a differential targeting analysis between Husk and V2-IST tissues. First, we must reformat the data for use with limma and chose a *p*-value significance threshold for the differential analysis. We can then proceed to run the differential targeting analysis.

In [ ]:
#limma
#set threshold for p-values
thres <- 0.01

#prepare data for limma
tissues <- samples.annot[rownames(ind_net_log_tab), "tissue"]
limmadiffexpr <- list()
limmadiffexprsignif <- list()
tab <- ind_net_log_tab[tissues %in% c("Husk", "V2-IST"),]
tissues_list <- tissues[tissues %in% c("Husk", "V2-IST")]

#create a binary vector with 0 representing HUSK samples and 1 representing V2-IST samples # nolint
tissue <- rep(0, nrow(tab))
tissue[tissues_list == "V2-IST"] <- 1

# Run limma analysis
design <- model.matrix(~ tissue)
fit <- lmFit(t(tab), design = design)
fit <- eBayes(fit)


We extract the significant results for Husk and V2-IST, and print the number of differentially targeted genes and the number of TFs targeting these genes. We see that there are 2123 genes that are more targeted in Husk than V2-IST by 55 TFs, and 2075 genes that are more targeted by 54 TFs in V2-IST than in Husk. 

In [ ]:
# Extract results
limmadiffexpr[["V2"]] <- topTable(fit, number = ncol(tab))
limmadiffexpr[["husk"]] <- topTable(fit, number = ncol(tab))

# extract results below the significance threshold
limmadiffexprsignif[["V2"]] <- limmadiffexpr[["V2"]][limmadiffexpr[["V2"]]$adj.P.Val <= thres & #nolint
                                    limmadiffexpr[["V2"]]$logFC > 0, ]
limmadiffexprsignif[["husk"]] <- limmadiffexpr[["husk"]][limmadiffexpr[["husk"]]$adj.P.Val <= thres & #nolint
                                    limmadiffexpr[["husk"]]$logFC < 0, ]
                                            
#extracting differentially targeted genes
a <- lapply(limmadiffexprsignif, rownames)
diff_genes <- lapply(a, function(x){unique(unlist(lapply(x, function(y) strsplit(y, "_")[[1]][1])))})
nb_diff_genes <- unlist(lapply(diff_genes, function(x) length(unique(x)))) #number of differentially targeted genes in each tissue
print("Differentially targeted genes:")
nb_diff_genes

#extracting differentially targeting TFs
diff_TFs <- lapply(a, function(x){unlist(lapply(x, function(y) strsplit(y, "_")[[1]][2]))})
nb_diff_TFs <- unlist(lapply(diff_TFs, function(x) length(unique(x)))) #number of differentially targeted genes in each tissue
print("Differentially targeting TFs:")
nb_diff_TFs


After identifying the differentially targeted genes, we can look for functional enrichment among them. To do this, we will perform a Gene Ontology (GO) enrichment analysis. To account for the nested structure of the Gene Ontology, we used the elim algorithm from the R Bioconductor topGO package, and the Fisher’s exact test to test for enrichment significance. Because we will perform GO enrichment analysis again later on, we write a function that performs it. The function takes as input the following:
* ***geneNames*** - gene names from the GO annotation
* ***list.topgenes*** - differentially expressed genes
* ***all.genes*** - list of genes in our data
* ***geneID2GO*** - the mapping of the GO genes to the GO annotation
* ***max.nb.genes*** - the maximum number of genes to show
* ***tag*** - the tissue of interest
* ***plots*** - logical; indicates whether to save plots of enriched GO terms

In [ ]:
#function for GO
GOenrich <- function(geneNames, list.topgenes, all.genes, geneID2GO, max.nb.genes, tag, plots = FALSE) {
  allRes <- list()
  allRes[[tag]] <- list()
  GOtermsGenes <- list()
  GOtermsGenes[[tag]] <- list()

  for(i in names(list.topgenes)) {
    allRes[[tag]][[i]] <- list()
    if(length(list.topgenes[[i]]) >= 5) {
      myInterestingGenes <- unique(list.topgenes[[i]]) #nolint
      geneList <- factor(as.integer(geneNames[geneNames %in% all.genes] %in% myInterestingGenes)) #nolint
      names(geneList) <- geneNames[geneNames %in% all.genes]
      GOtermsGenes[[tag]][[i]] <- data.frame("Ontology" = character(0),
                                              "GOTerm" = character(0),
                                              "Genes" = character(0))
      for(ont in c("BP", "MF")) {
        GOdata <- new("topGOdata", ontology = ont, allGenes = geneList,
                    annot = annFUN.gene2GO, gene2GO = geneID2GO, nodeSize = 5)
        ## Compute stats
        test.stat <- new("elimCount", testStatistic = GOFisherTest, name = "Fisher test")#nolint
        resultElimFisher <- getSigGroups(GOdata, test.stat)
        allRes[[tag]][[i]][[`ont`]] <- GenTable(GOdata, elim = resultElimFisher, orderBy = "elim", #nolint
                                           ranksOf = "elim", topNodes = max.nb.genes) #nolint     
        allRes[[tag]][[i]][[`ont`]]$ontology <- rep(ont, nrow(allRes[[tag]][[i]][[`ont`]]))#nolint
        
        #plot
        if(plots == TRUE) {
          pdf(file = paste0("TopGONodes_", ont, ".pdf"))
          showSigOfNodes(GOdata, score(resultElimFisher), firstSigNodes = 10, useInfo = 'all')#nolint
          dev.off()
        }
        
        
        myterms <- allRes[[tag]][[i]][[`ont`]]$GO.ID
        mygenes  <- genesInTerm(GOdata, myterms)
        
        for (j in 1:length(myterms)) {
          myterm <- myterms[j]
          mygenesforterm <- mygenes[myterm][[1]]
          myfactor <- mygenesforterm %in% myInterestingGenes # find the genes that are in the list of genes of interest #nolint
          mygenesforterm2 <- mygenesforterm[myfactor == TRUE]
          mygenesforterm2 <- paste(mygenesforterm2, collapse = ",")
          GOtermsGenes[[tag]][[i]] <- rbind(GOtermsGenes[[tag]][[i]],
                                    data.frame("Ontology" = ont, "GOTerm" = myterm, "Genes" = mygenesforterm2))#nolint
        }
      }
      allRes[[tag]][[i]]=rbind(allRes[[tag]][[i]]$BP, allRes[[tag]][[i]]$MF)
    }
  }
  return(list(allRes, GOtermsGenes))
}

Now that we have this function, we can perform the GO enrichment analysis on differentially targeted genes from each tissue. We sort the results by *p*-values and we see that among the most significant enriched terms for Husk, some are related to leaf growth and differentiation (cell growth, vacuole organization, root hair cell differentiation), protein activity and post translational protein modifications (protein serine/threonine kinase activity, protein dimerization activity, protein glycosylation, protein targeting to vacuole), and signal transduction (magnesium ion transport, signal transduction). For the V2-IST tissue, we find an enrichment in processes related to cell division (cytokinesis by cell plate formation,  regulation of G2/M transition of mitotic cell cycle, spindle assembly, cell population proliferation), leaf morphogenesis and photosynthesis (carotenoid biosynthetic process, photosynthesis, chlorophyll binding, photosynthetic electron transport chain) and gene expression regulation (gene silencing). 

In [ ]:
if(precomputed==0){
    #prepare GO annotation
    all_genes=unique(unlist(lapply(colnames(ind_net_log_tab), function(x){strsplit(x, "_")[[1]][1]})))
    ZmB73_5a_xref.GO.topGO.FG <- read.delim(GO.file, header=FALSE, stringsAsFactors = F)
    geneNames<-ZmB73_5a_xref.GO.topGO.FG$V1
    geneID2GO <- readMappings(file = GO.file)
   
    #perform GO enrichment analysis
    diff_GO_res <- GOenrich(geneNames = geneNames, geneID2GO = geneID2GO,
                        list.topgenes = diff_genes, all.genes = all_genes,
                        tag = "V2", max.nb.genes = 100)
} else {
    load(paste0(ppath, "GO_diff_targeted_genes.Rdata"))
}

#extract tissue specific enrichment
husk <- diff_GO_res[[1]][[1]]$husk
V2 <- diff_GO_res[[1]][[1]]$V2

#order enriched terms by most significant hits
husk <- husk[order(husk$elim), ]
V2 <- V2[order(V2$elim), ]
cat("Husk:\n")
head(husk, n = 15)
cat("V2-IST:\n")
head(V2, n = 15)


# Identification of top enhancer targets

We can also identify the top target gene for each enhancer. This is done as follows: for each enhancer-gene pair, we compute an average targeting score. To this end, we extract all of the TFs with predicted binding sites in an enhancer. We then compute the average of the edge weight between each TF and one target gene. The gene with the highest average targeting score is the most likely target.

*  This section needs as input:
    + A data frame with samples annotations (samples.annot);
    + A data frame with the lioness log-transformed edges (ind_net_log_tab)
    + A data frame linking the predicted TFBS, the enhancers, and their potential target genes (enh.genes).
    

In [ ]:
load(paste0(ppath, "enhancers_genes.Rdata"))
tis <- samples.annot$tissue 

enh <- sort(unique(c(enh.genes$enh))) # Extract list of enhancers
edges <- t(ind_net_log_tab[rownames(samples.annot),])
edges <- as.data.frame(edges[,tis %in% c("Husk", "V2-IST")]) # Create edges table

### Functions
## For each enhancer-gene pair, compute average TF-gene edge value across all TF with predicted binding site in the enhancer
average.edge <- function(geneID, tfbs, edges, gene) {#nolint
    tfs <- tfbs$Alias[tfbs$geneID == geneID]
    res <- NULL
    for(tf in tfs) {
        alltfs <- unlist(strsplit(tf, ";"))
        e <- edges[paste(geneID, alltfs, sep = "_"), ]
        res <- rbind(res, colSums(e, na.rm = T) / nrow(e))
    }
    if(is.null(res)) { return(rep(0,12)) } else { return(colSums(res) / nrow(res)) }#nolint
}

## Find enhancer-gene pairs and compute average edge for each of them
compute.average.edge <- function(enhID, pairs, edges) {#nolint
  tmp <- pairs[pairs$enh == enhID, ]
  g <- unique(tmp$geneID)
  if(length(g) > 0 & !all(is.na(g))) {
    score.genes <- t(apply(data.frame(g, stringsAsFactors = F), 1, average.edge, 
                           tfbs = tmp, edges = edges))
        rownames(score.genes) <- g
    if(nrow(score.genes) == 1) {
      avg.husk <- mean(score.genes[, 1:6])
      avg.V2 <- mean(score.genes[, 7:12])
      names(avg.husk) <- names(avg.V2) <- g
    } else {
      avg.husk=rowSums(score.genes[,1:6])/ncol(score.genes[,1:6])
      avg.V2=rowSums(score.genes[,7:12])/ncol(score.genes[,7:12])
    }
    out <- data.frame("enh" = rep(enhID, length(avg.husk)), "gene" = names(avg.husk), #nolint
                       "edge.husk" = avg.husk, "top.husk"=ifelse(avg.husk == max(avg.husk), 1, 0), #nolint
                       "edge.V2" = avg.V2, "top.V2" = ifelse(avg.V2 == max(avg.V2), 1, 0),#nolint
                       stringsAsFactors = F)
  } else {
    out <- data.frame("enh" = enhID, "gene" = NA, "edge.husk" = NA, "top.husk" = NA,
                     "edge.V2" = NA, "top.V2" = NA, stringsAsFactors = F)
  }
  return(out)
}

### Compute targeting for all enhancer-gene pairs and pick highest target
if(precomputed==0){
    enh.gene.pairs <- t(apply(data.frame(enh, stringsAsFactors=F), 1, compute.average.edge, 
                          pairs = enh.genes, edges = edges)) #nolint
} else {
    load(paste0(ppath, "enhancer_gene_pairs.Rdata"))
}

##Make result table
res <- NULL
for(i in 1:length(enh.gene.pairs)) {
  res <- rbind(res, enh.gene.pairs[[i]])
}
rownames(res) <- NULL
head(res[res$top.husk ==1 | res$top.V2==1,],15)

# Identification of tissue-specific modules with CONDOR & ALPACA

To further characterize differential targeting between Husk and V2-IST, we will identify tissue-specific communities in our regulatory networks using CONDOR and ALPACA. First, we will compute the average LIONESS edges across six replicates for each tissue, as it is the required input for ALPACA.

* This section needs as input:
    + A data frame with the lioness log-transformed edges (ind_net_log_tab)
    + The original panda and lioness outputs (panda, lioness)
    + A table with information on transposable elements and enhancer overlaps, and the potential target genes.

In [ ]:
#get average edges by tissue
tmp <- by(ind_net_log_tab, samples.annot[rownames(ind_net_log_tab), "tissue"],
                                function(x){colSums(x)/nrow(x)})
edges_by_tissue <- data.frame(matrix(unlist(tmp),
                             ncol=length(unique(samples.annot[rownames(ind_net_log_tab), "tissue"])))) #nolint
colnames(edges_by_tissue) <- names(tmp)
rownames(edges_by_tissue) <- names(tmp[[1]])
rm(tmp)

Now we can identify communities in each tissue-specific network with CONDOR. 

In [ ]:
load(paste0(ppath, "corres_TF_athaliana_zmays.Rdata"))
### Compute tissue-specific condor networks
# prepare condor input - extract edges for TFs that are expressed in at least one of the tissues
maize_TF <- ortho[order(ortho$TF),]
expr_tfs <- unique(maize_TF$TF[maize_TF$zmays_eg_homolog_ensembl_gene %in% rownames(expr)])
core_tab <- lapply(rownames(edges_by_tissue), function(x) { return(unlist(strsplit(x, "_")[[1]]))}) #nolint
core_tab <- data.frame(matrix(unlist(core_tab), ncol=2, byrow=TRUE),
                       stringsAsFactors = FALSE)
colnames(core_tab) <- c("Gene", "TF")
core_tab <- core_tab[core_tab$TF %in% expr_tfs, ]

# run condor
condor_network <- list()
for (i in colnames(edges_by_tissue)[colnames(edges_by_tissue) %in% c("Husk", "V2-IST")]) {
  cat("Finding community structure for tissue ", i, "\n")
  tab <- cbind(core_tab, "edge" = edges_by_tissue[paste(core_tab$Gene, core_tab$TF, sep = "_"), i]) #nolint
  tab <- tab[tab$edge != Inf, ]
  condor_network[[`i`]] <- createCondorObject(tab, return.gcc=T)
  condor_network[[`i`]] <- condorCluster(condor_network[[`i`]], project = F) #nolint
}

###Stats on networks
nb_com <- sapply(condor_network, function(x) nrow(x$Qcoms)) #number of communities #nolint
modularity <- sapply(condor_network, function(x) max(x$modularity)) #modularity
cat("\nNumber of communities by network:\n")
nb_com
cat("\nNetworks modularities:\n")
modularity

Next, we will use ALPACA to compare the network structures of the two tissues. We first need to make a slight modification to the main alpaca function.

In [ ]:
#modified ALPACA function
alpaca <- function (net.table, file.stem, verbose = T) {
  net.table[, 1] <- paste(net.table[, 1], "A", sep = "_")
  net.table[, 2] <- paste(net.table[, 2], "B", sep = "_")
  print("Detecting communities in control network...")
  ctrl.pos <- net.table[net.table[, 3] >= 0, 1:3]
  ctrl.elist <- data.frame(red = ctrl.pos[, 2], blue = ctrl.pos[, 
                                                                1], weights = ctrl.pos[, 3])
  ctrl.condor <- createCondorObject(ctrl.elist)
  ctrl.condor <- condorCluster(ctrl.condor, project = F)
  ctrl.memb <- c(ctrl.condor$red.memb[, 2], ctrl.condor$blue.memb[, 
                                                                  2])
  names(ctrl.memb) <- c(as.character(ctrl.condor$red.memb[, 
                                                          1]), as.character(ctrl.condor$blue.memb[, 1]))
  if (!(is.null(file.stem))) 
    write.table(ctrl.memb, paste(c(file.stem, "_ALPACA_ctrl_memb.txt"), 
                                 collapse = ""), row.names = T, col.names = F, quote = F, 
                sep = "\t")
  pos.table <- net.table[intersect(which(net.table[, 3] >= 
                                           0), which(net.table[, 4] >= 0)), ]
  pos.graph <- graph.edgelist(as.matrix(pos.table[, 1:2]), 
                              directed = T)
  if (length(setdiff(V(pos.graph)$name, names(ctrl.memb))) > 
      0) {
    uncounted <- setdiff(V(pos.graph)$name, names(ctrl.memb))
    unc.memb <- sample(1:max(ctrl.memb), length(uncounted), 
                       replace = T)
    names(unc.memb) <- uncounted
    ctrl.memb <- c(ctrl.memb, unc.memb)
  }
  print("Computing differential modularity matrix...")
  dwbm <- alpacaComputeDWBMmatMscale(pos.table, ctrl.memb[V(pos.graph)$name])
  if (verbose) {
    write.table(dwbm, paste(c(file.stem, "_DWBM.txt"), collapse = ""), 
                row.names = T, col.names = T, quote = F, sep = "\t")
    write.table(rownames(dwbm), paste(c(file.stem, "_DWBM_rownames.txt"), 
                                      collapse = ""), row.names = T, col.names = T, quote = F, 
                sep = "\t")
    write.table(colnames(dwbm), paste(c(file.stem, "_DWBM_colnames.txt"), 
                                      collapse = ""), row.names = T, col.names = T, quote = F, 
                sep = "\t")
  }
  ntfs <- nrow(dwbm)
  ngenes <- ncol(dwbm)
  this.B <- array(0, dim = c(ntfs + ngenes, ntfs + ngenes))
  this.B[1:ntfs, (ntfs + 1):(ntfs + ngenes)] <- dwbm
  this.B[(ntfs + 1):(ntfs + ngenes), 1:ntfs] <- t(dwbm)
  print("Computing differential modules...")
  louv.memb <- alpaca.genlouvain(this.B)
  names(louv.memb) <- c(rownames(dwbm), colnames(dwbm))
  print("Computing node scores...")
  louv.Ascores <- NULL
  louv.Bscores <- NULL
  for (i in 1:max(louv.memb)) {
    this.comm <- names(louv.memb)[louv.memb == i]
    this.tfs <- this.comm[grep("_A", this.comm)]
    this.genes <- this.comm[grep("_B", this.comm)]
    if ((length(this.tfs) > 1 ) & (length(this.genes) > 1)) {
      tf.sums <- apply(dwbm[this.tfs, this.genes], 1, sum)
      gene.sums <- apply(dwbm[this.tfs, this.genes], 2, 
                         sum)
    }        else {
      tf.sums <- sum(dwbm[this.tfs, this.genes])
      gene.sums <- dwbm[this.tfs, this.genes]
    }
    this.denom <- sum(dwbm[this.tfs, this.genes])
    louv.Ascores <- c(louv.Ascores, tf.sums/this.denom)
    louv.Bscores <- c(louv.Bscores, gene.sums/this.denom)
  }
  louv.scores <- c(louv.Ascores, louv.Bscores)
  if (!is.null(file.stem)) {
    write.table(cbind(names(louv.memb), as.vector(louv.memb)), 
                paste(c(file.stem, "_ALPACA_final_memb.txt"), collapse = ""), 
                col.names = F, row.names = F, quote = F, sep = "\t")
    write.table(cbind(names(louv.scores), louv.scores), paste(c(file.stem, 
                                                                "_ALPACA_scores.txt"), collapse = ""), row.names = F, 
                col.names = F, quote = F, sep = "\t")
  }
  list(louv.memb, louv.scores)
}

ALPACA allows us to compare two networks and find differences in their community structure. We can use the network structure we identified with CONDOR to explore differences in the regulatory networks of the Husk and V2-IST tissues. This will allow us to identify genes and TFs that contributed to changes in the modularity between the two tissues. ALPACA considers one network as a background "control"; to account for this, we run ALPACA two times, using in turn the Husk and V2-IST networks as the background and we extract the top genes for each tissue. This can take a few minutes.

When setting V2-IST as reference, we find 75 modules in the husk network, including 9 with at least 10 genes and one TF.

When setting Husk as reference, we find 70 modules in the V2-IST network, including 11 with at least 10 genes and one TF.

In [ ]:
#set up alpaca input object
core_tab_alpaca <- core_tab[,2:1]
max_nb_genes <- 100
TFs <- paste0("tf", 1:length(unique(core_tab$TF)))
names(TFs) <- unique(core_tab$TF)
genes <- paste0("gene", 1:length(unique(core_tab$Gene)))
names(genes) <- unique(core_tab$Gene)

core_tab_alpaca <- core_tab
core_tab_alpaca$Gene <- genes[core_tab_alpaca$Gene]
core_tab_alpaca$TF <- TFs[core_tab_alpaca$TF]

#alpaca on husk - set V2-IST as reference and compute the differential modularity between husk and V2-IST. 
set.seed(1)
V2_husk_edges <- data.frame(core_tab[, 2:1],
                        "V2" = edges_by_tissue[paste(core_tab$Gene, core_tab$TF, sep = "_"), "V2-IST"], #nolint
                        "husk" = edges_by_tissue[paste(core_tab$Gene, core_tab$TF, sep = "_"), "Husk"], stringsAsFactors = FALSE)#nolint
if (precomputed==0){
    V2_husk <- alpaca(V2_husk_edges, file.stem = paste0("alpaca.results.V2ref"), verbose=TRUE)#nolint
} else {
    load(paste0(ppath, "alpaca_results.Rdata"))
}

#List of genes in each community of husk network
husk_genes <- tapply(names(V2_husk[[1]]), V2_husk[[1]], function(x) { gsub("_B", "", x[grep("_B", x)])})#nolint
#List of TFs in each community of husk network
husk_TFs <- tapply(names(V2_husk[[1]]), V2_husk[[1]], function(x) { gsub("_A", "", x[grep("_A", x)])})#nolint

#List of genes in each community of husk network, with their score reflecting how much of the differential modularity they explain 
husk_genes_scores <- lapply(husk_genes, function(x, scores) {
  res <- scores[paste0(x, "_B")]
  names(res) <- x
  return(sort(res))
}, scores = V2_husk[[2]])

#List of TFs in each community of husk network, with their score reflecting how much of the differential modularity they explain 
husk_TFs_scores <- lapply(husk_TFs, function(x, scores) {
  res <- scores[paste0(x, "_A")]
  names(res) <- x
  return(sort(res))
}, scores = V2_husk[[2]])

#List of 100 top genes
husk_topgenes <- lapply(husk_genes_scores, function(x) {
  n <- names(sort(x, decreasing = T))[1:max_nb_genes]
  return(n[!is.na(n)])})
names(husk_topgenes) <- paste0("Husk_", names(husk_topgenes))

#alpaca on V2-IST - set husk as reference and compute the differential modularity between husk and V2-IST. 
husk_V2_edges <- data.frame(core_tab[, 2:1],
                        "husk" = edges_by_tissue[paste(core_tab$Gene, core_tab$TF, sep = "_"), "Husk"], #nolint
                        "V2" = edges_by_tissue[paste(core_tab$Gene, core_tab$TF, sep = "_"), "V2-IST"], stringsAsFactors = FALSE)#nolint
if (precomputed==0){
    husk_V2 <- alpaca(husk_V2_edges, file.stem = "alpaca.results.huskref", verbose = TRUE)#nolint
}

#List of genes in each community of V2-IST network
V2_genes <- tapply(names(husk_V2[[1]]), husk_V2[[1]], function(x) { gsub("_B", "", x[grep("_B", x)])})#nolint

#List of TFs in each community of V2-IST network
V2_TFs <- tapply(names(husk_V2[[1]]), husk_V2[[1]], function(x) { gsub("_A", "", x[grep("_A", x)])})#nolint

#List of genes in each community of V2-IST network, with their score reflecting how much of the differential modularity they explain 
V2_genes_scores <- lapply(V2_genes, function(x, scores) {
  res <- scores[paste0(x, "_B")]
  names(res) <- x
  return(sort(res))
}, scores = husk_V2[[2]])

#List of TFs in each community of V2-IST network, with their score reflecting how much of the differential modularity they explain 
V2_TFs_scores <- lapply(V2_TFs, function(x, scores) {
  res <- scores[paste0(x, "_A")]
  names(res)=x
  return(sort(res))
}, scores = husk_V2[[2]])

#List of 100 top genes
V2_topgenes <- lapply(V2_genes_scores, function(x) {
  n <- names(sort(x, decreasing = T))[1:max_nb_genes]
  return(n[!is.na(n)])})
names(V2_topgenes) <- paste0("V2-IST_", names(V2_topgenes))

To compare the gene content of the communities in the Husk and V2-IST networks, we calculate the jaccard index of each pair of communities. The best corresponding community is the one with the highest jaccard index. If the best jaccard index is lower than 0.4, we consider a community as tissue-specific, otherwise as shared.

We identify 2 tissue-specific communities in the Husk network, and 3 in the V2-IST network. We also identify 7 shared communities. 

In [ ]:
#compute jaccard index
jaccard <- NULL
for(i in 1:length(V2_genes)) {
  tmp <- c()
  for(j in 1:length(husk_genes)) {
    tmp <- c(tmp, (sum(V2_genes[[i]] %in% husk_genes[[j]]) / length(union(V2_genes[[i]], husk_genes[[j]]))))#nolint
  }
  jaccard <- cbind(jaccard, tmp)
}
rownames(jaccard) <- paste("Husk", 1:nrow(jaccard), sep = "_")
colnames(jaccard) <- paste("V2-IST", 1:ncol(jaccard), sep = "_")

#extract tissue-specific communities
husk_jaccard <- apply(jaccard, 1, max)[unlist(lapply(husk_TFs, length)) >= 1 & unlist(lapply(husk_genes, length)) >= 10] #nolint
husk_unique <- husk_jaccard[husk_jaccard<0.4]
cat("Husk top jaccard index for each community:\n")
husk_jaccard

V2_jaccard <- apply(jaccard, 2, max)[unlist(lapply(V2_TFs, length)) >=1 & unlist(lapply(V2_genes, length)) >= 10]#nolint
V2_unique <- V2_jaccard[V2_jaccard<0.4]
cat("V2-IST top jaccard index for each community:\n")
V2_jaccard

## Extract list of topgenes 
find.unique.husk <-function(i, h = husk_genes_scores) {
  names(head(sort(h[[i]], decreasing = T), max_nb_genes))
}
find.unique.V2 <- function(i, v = V2_genes_scores){
    names(head(sort(v[[i]], decreasing = T), max_nb_genes))
}

husk_unique_topgenes <- subset(husk_topgenes, names(husk_topgenes) %in% names(husk_unique))

V2_unique_topgenes <- subset(V2_topgenes, names(V2_topgenes) %in% names(V2_unique))

shared_topgenes <- subset(husk_topgenes, (!(names(husk_topgenes) %in% names(husk_unique)) & (names(husk_topgenes) %in% names(husk_jaccard))))

Once again, we perform Gene Ontology categories enrichment analysis among the top genes of each tissue-specific community for Husk and V2-IST networks, and shared communities between husk and V2-IST.
The first Husk-specific community is enriched for functions linked to cell wall organization or biogenesis. The second one is enriched for functions linked to rRNA-containing ribonucleoprotein complex binding and oligosaccharide biosynthetic process.

The first V2-IST-specific community is enriched for several functions linked to response to hormones (regulation of jasmonic acid mediated signal; cytokinin-activated signaling pathway); regulation of transcription (regulation of nucleic acid-templated transcription; transcription corepressor activity), and amino-acid biosynthetic processes (sulfur compound biosynthetic process; aromatic amino acid family biosynthetic process). The second V2-IST community is enriched in histone modification (cellular macromolecule biosynthetic process, histone methyltransferase activity) and mitotic process (sister chromatid cohesion). Finally, the third V2-IST-specific community is enriched in leaf vascular tissue pattern formation and response to hypoxia processes. All these functions are coherent with a dividing, differentiating seedling tissues.


In [ ]:
### Compute Gene Ontology enrichment analyses for each tissue-specufic regulatory community.
if(precomputed==0){
    ## V2-specific
    V2_GO_res <- GOenrich(geneNames = geneNames, geneID2GO = geneID2GO,
         list.topgenes = V2_unique_topgenes, all.genes = unique(unlist(V2_genes)), tag = "V2-specific_communities",  max.nb.genes = 100)
    ## Husk-specific
    Husk_GO_res <- GOenrich(geneNames = geneNames, geneID2GO = geneID2GO,
         list.topgenes = husk_unique_topgenes, all.genes = unique(unlist(husk_genes)), tag = "Husk-specific_communities",  max.nb.genes = 100)
} else {
    load(paste0(ppath, "GO_tissue_specific_communities.Rdata"))
}

#Order enriched terms by most significant hits
cat("V2-IST specific communities:\n")
for(i in 1:length(V2_GO_res[[1]][[1]])){
    tmp <- V2_GO_res[[1]][[1]][[i]]
    tmp <- tmp[order(tmp$elim), ]
    cat("\n", names(V2_GO_res[[1]][[1]])[i], ":\n", sep="")
    print(head(tmp[tmp$elim<=0.01,], n = 15))
}

cat("Husk specific communities:\n")
for(i in 1:length(Husk_GO_res[[1]][[1]])){
    tmp <- Husk_GO_res[[1]][[1]][[i]]
    tmp <- tmp[order(tmp$elim), ]
    cat("\n", names(Husk_GO_res[[1]][[1]])[i], ":\n", sep="")
    print(head(tmp[tmp$elim<=0.01,], n = 15))
}

# Enrichment analysis for transposable elements families and superfamilies among enhancers and TFBS

We can further analyse the gene regulatory networks and investigate the role of TEs in the tissue-specific regulation of gene expression. To this end, we look for relative enrichment of TE superfamilies and orders in tissue-specific enhancers and/or TFBS, and analyse the functions of the genes they regulate. But first, we investigate the proportion of active enhancers that are overlapping at least one TE in Husk and V2-IST, and search for enrichment in specific order of TEs (LTR, LINE, TIR, MITE, or Helitrons). The TE annotation of B73 was generated using RepeatMasker and an updated library of TE can be found here: https://github.com/oushujun/EDTA/blob/master/database/maizeTE11122019.

We find that each category of enhancers is enriched in TE from different orders, suggesting that different types of TE were domesticated to build regulatory networks, both shared between tissues and tissue-specific. Husk-specific activated enhancers are enriched in Miniature inverted-repeat transposable elements (MITEs) while V2-IST-specific activated enhancers are not enriched in any specific TE order. Finally, enhancers active in both tissues are enriched in Long Terminal Repeats (LTRs).

In [ ]:
### Load and preprocess data
load(paste0(ppath, "enhancers_TE_overlaps.Rdata"))
enh.TE <- enh.TE[enh %in% enh.genes$enh]
tissue.enh <- unique(enh.TE[, c("enh.pos", "tissue")])$tissue
names(tissue.enh) <- unique(enh.TE[, c("enh.pos", "tissue")])$enh.pos

### For each enhancer, determine if it is overlapping TE from zero ("None"), one or multiple orders
res=unlist(by(enh.TE, enh.TE$enh.pos, function(annot){
    annot.order=unique(annot$order)
    if(length(annot.order[!(annot.order %in% "None")])>1){
        annot.order <- "Multiple"
    }
    if(length(annot.order[!(annot.order %in% "None")])==1){
        annot.order <- annot.order[!(annot.order %in% "None")]
    }
    return(annot.order)
}))

### For each category of enhancers, compute the proportion of enhancers overlapping TEs from each order
colors.TE <- c("white", "gold", "forestgreen", "green", "blue", "cyan", "purple")
names(colors.TE) <- c("None", "Multiple", "LTR", "LINE", "TIR", "MITE", "Helitron")
husk.enh.TE <- summary(factor(res[(tissue.enh[names(res)] %in% "Husk")], level=unique(as.character(res))))
V2.enh.TE <- summary(factor(res[(tissue.enh[names(res)] %in% "V2-IST")], level=unique(as.character(res))))
Both.enh.TE <- summary(factor(res[(tissue.enh[names(res)] %in% "Both")], level=unique(as.character(res))))
All.enh.TE <- summary(factor(res, level=unique(as.character(res))))
prop=cbind("All"=All.enh.TE[names(colors.TE)]/sum(All.enh.TE),
           "Shared"=Both.enh.TE[names(colors.TE)]/sum(Both.enh.TE),
           "Husk"=husk.enh.TE[names(colors.TE)]/sum(husk.enh.TE), 
           "V2-IST"=V2.enh.TE[names(colors.TE)]/sum(V2.enh.TE))

### Compute corresponding Odds ratio and p-values using fisher tests
res.enrich.TE.counts <- NULL
for(te in names(colors.TE)[!(names(colors.TE) %in% c("None", "Multiple"))]){
    real.tmp <- tapply(enh.TE$order, enh.TE$enh.pos, function(x){any(x == te)})
    fh <- fisher.test(table(real.tmp, (tissue.enh[names(real.tmp)]=="Husk")))
    fv <- fisher.test(table(real.tmp, (tissue.enh[names(real.tmp)]=="V2-IST")))
    fb <- fisher.test(table(real.tmp, (tissue.enh[names(real.tmp)]=="Both")))
    res.enrich.TE.counts <- rbind(res.enrich.TE.counts, 
                               c(fh$p.value, fh$estimate,
                                 fv$p.value, fv$estimate,
                                 fb$p.value, fb$estimate))
}
rownames(res.enrich.TE.counts) <- names(colors.TE)[!(names(colors.TE) %in% c("None", "Multiple"))]
colnames(res.enrich.TE.counts) <- c("p-value Husk", "OR Husk", "p-value V2-IST", "OR V2-IST", "p-value Shared", "OR Shared")
res.enrich.TE.counts

### Plot the result as a barplot
par(mar=c(4,7,7,2)+.1, las=1)
barplot(prop, horiz=T, col=colors.TE, cex.axis=1.5, cex.names=1.8, cex.lab=1.8, xlab="Proportion of enhancers overlapping TEs")
par("xpd"=TRUE)
legend((par("usr")[1]-(par("usr")[2]-par("usr")[1])/4), (par("usr")[4]+(par("usr")[4]-par("usr")[3])/4), 
       legend=names(colors.TE), 
       fill=colors.TE,
       bty="n", ncol=3, cex=1.8)
par("xpd"=FALSE)


Then, we investigate the proportion of TFBS predicted in husk-specific enhancers that are overlapping at least one TE.  

We find that husk-specific TFBS do not overlap more frequently than expected by chance any particular TE order.

In [ ]:
load(paste0(ppath, "tfbs_enhancers_TE_genes.Rdata"))

# Extract tfbs and TE information from table
tfbs.TE = tfbs.TE.genes
tfbs.TE[, geneID := NULL]
tfbs.TE[, gene.tss := NULL]
tfbs.TE <- unique(tfbs.TE)

### Test for TE order enrichment among husk TFBS
res.enrich.TFBS.husk.order=NULL
tissue <- tfbs.TE$tissue
for(te in c("TIR", "MITE", "LTR", "LINE")){
  real.tmp <- (tfbs.TE$order == te)
  fh <- fisher.test(table(real.tmp, (tissue=="Husk")))
  res.enrich.TFBS.husk.order=rbind(res.enrich.TFBS.husk.order, 
                             c(fh$p.value, fh$estimate))
  }
rownames(res.enrich.TFBS.husk.order) <- c("TIR", "MITE", "LTR", "LINE")
colnames(res.enrich.TFBS.husk.order) <- c("p-value Husk", "OR Husk")

res.enrich.TFBS.husk.order

We perform the same analysis and find that TFBS predicted in V2-IST-specific enhancers preferentially overlap Terminal Inverted Repeat sequences (TIR). We then investigate if any specific superfamily of TIR is responsible for this enrichment, and find that they are enriched in TIR Mutator (DTM) elements.

In [ ]:
### Test for TE order enrichment among TFBS
res.enrich.TFBS.V2.order=NULL
tissue <- tfbs.TE$tissue
for(te in c("TIR", "MITE", "LTR", "LINE")){
  real.tmp <- (tfbs.TE$order == te)
  fh <- fisher.test(table(real.tmp, (tissue=="V2-IST")))
  res.enrich.TFBS.V2.order=rbind(res.enrich.TFBS.V2.order, 
                             c(fh$p.value, fh$estimate))
  }
rownames(res.enrich.TFBS.V2.order) <- c("TIR", "MITE", "LTR", "LINE")
colnames(res.enrich.TFBS.V2.order) <- c("p-value V2-IST", "OR V2-IST")

res.enrich.TFBS.V2.order

### Test for TE superfamily enrichment among TFBS
res.enrich.TFBS.V2.sf <- NULL
TE.order.sf <- unique(paste(tfbs.TE$order, tfbs.TE$superfamily, sep="_"))
tissue <- tfbs.TE$tissue
for(te in c("TIR_DTA", "TIR_DTC", "TIR_DTM")){
  real.tmp <- (paste(tfbs.TE$order, tfbs.TE$superfamily, sep="_")== te)
  fh <- fisher.test(table(real.tmp, (tissue=="V2-IST")))
  res.enrich.TFBS.V2.sf <- rbind(res.enrich.TFBS.V2.sf, 
                              c(fh$p.value, fh$estimate, 
                                fh$conf.int[1], fh$conf.int[2]))
}
rownames(res.enrich.TFBS.V2.sf) <- c("TIR_DTA", "TIR_DTC", "TIR_DTM")
colnames(res.enrich.TFBS.V2.sf) <- c("p-value V2-IST", "OR V2-IST", "CI Inf V2-IST",  "CI Sup V2-IST")

res.enrich.TFBS.V2.sf

#plotting results for superfamilies
res.enrich.TFBS.V2.sf <- data.frame(res.enrich.TFBS.V2.sf[3:1,])
color <- c(rep("forestgreen",3) )

par(las=1, mar=c(4,7,0,0)+.1)
plot(log2(res.enrich.TFBS.V2.sf$OR.V2.IST), c(1:nrow(res.enrich.TFBS.V2.sf)), 
     col=color, pch=18, cex=3, xlab="Odds ratio", ylab="", yaxt="n", ylim=c(0.5,3.5),
     xlim=c(min(log2(res.enrich.TFBS.V2.sf$CI.Inf.V2.IST)), max(log2(res.enrich.TFBS.V2.sf$CI.Sup.V2.IST))), 
     cex.axis=1.5, cex.lab=1.8)
for(i in 1:3){
  lines(c(log2(res.enrich.TFBS.V2.sf$CI.Inf.V2.IST[i]), log2(res.enrich.TFBS.V2.sf$CI.Sup.V2.IST[i])), c(i,i), 
      col=color[i], lwd=3)
  lines(rep(log2(res.enrich.TFBS.V2.sf$CI.Inf.V2.IST[i]),2), c((i-0.1), (i+0.1)), 
        col=color[i], lwd=3)
  lines(rep(log2(res.enrich.TFBS.V2.sf$CI.Sup.V2.IST[i]),2), c((i-0.1), (i+0.1)) , 
        col=color[i], lwd=3)
}
lines(c(0,0), c(-1,8), lwd=3)
axis(side=2, labels=gsub("_", " ", rownames(res.enrich.TFBS.V2.sf)), at=1:3, cex.axis=1.8, cex.lab=2)

Finally, we look at the biological functions of the genes that are regulated by TFBS overlapping TIR Mutator elements. They are enriched in functions linked to cell growth (regulation of cell growth, cell wall polysaccharide biosynthetic process) and plant tissue differentiation (xylan biosynthetic process).

In [ ]:
load(paste0(ppath, "tfbs_enhancers_TE_genes.Rdata"))
if(precomputed==0){
    DTM.targets <- unique(sort(tfbs.TE.genes[order=="TIR" & superfamily=="DTM"]$geneID))
    TIR_GO_res <- GOenrich(geneNames = geneNames, geneID2GO = geneID2GO,
         list.topgenes = list("DTM"=DTM.targets), all.genes = unique(unlist(V2_genes)), tag = "TIR_targets",  max.nb.genes = 100)
} else {
    load(paste0(ppath, "GO_TIR_targets.Rdata"))
}
tmp <- TIR_GO_res[[1]][[1]]$DTM
tmp <- tmp[order(tmp$elim), ]
head(tmp, n = 15)


# References

1. <div class="csl-entry">Spitz, F. &#38; Furlong, E. E. M. (2012). Core promoter Transcription factors: from enhancer binding to developmental control. <i>NATURE REVIEWS | GENETICS</i>, <b>13</b>:613. https://doi.org/10.1038/nrg3207 </div>

2. <div class="csl-entry">Weber, B., Zicola, J., Oka, R., Stam, M. (2016). Plant Enhancers: A Call for Discovery. <i>Trends in Plant Science</i>, <b>21</b>(11):974–987. https://doi.org/10.1016/J.TPLANTS.2016.07.013 </div>

3. <div class="csl-entry">Zhu, B., Zhang, W., Zhang, T., Liu, B., Jiang, J. (2015). Genome-Wide Prediction and Validation of Intergenic Enhancers in Arabidopsis Using Open Chromatin Signatures. <i>The Plant Cell</i>, <b>27</b>(9):2415–2426. https://doi.org/10.1105/TPC.15.00537 </div>

4. <div class="csl-entry">Ricci, W. A., Lu, Z., Ji, L., Marand, A. P., Ethridge, C. L., Murphy, N. G., Noshay, J. M., Galli, M., Mejía-Guerra, M. K., Colomé-Tatché, M., Johannes, F., Rowley, M. J., Corces, V. G., Zhai, J., Scanlon, M. J., Buckler, E. S., Gallavotti, A., Springer, N. M., Schmitz, R. J., Zhang, X. (2019). Widespread long-range cis-regulatory elements in the maize genome. <i>Nature Plants</i>, <b>5</b>(12):1237–1249. https://doi.org/10.1038/S41477-019-0547-0 </div>

5. <div class="csl-entry">Lu, Z., Marand, A. P., Ricci, W. A., Ethridge, C. L., Zhang, X., Schmitz, R. J. (2019). The prevalence, evolution and chromatin signatures of plant regulatory elements. <i>Nature Plants</i>, <b>5</b>(12):1250–1259. https://doi.org/10.1038/S41477-019-0548-Z </div>

6. <div class="csl-entry">Li, E., Liu, H., Huang, L., Zhang, X., Dong, X., Song, W., Zhao, H., Lai, J. (2019). Long-range interactions between proximal and distal regulatory regions in maize. <i>Nature Communications</i>, <b>10</b>(1). https://doi.org/10.1038/S41467-019-10603-4 </div>

7. <div class="csl-entry">Oka, R., Zicola, J., Weber, B., Anderson, S. N., Hodgman, C., Gent, J. I., Wesselink, J. J., Springer, N. M., Hoefsloot, H. C. J., Turck, F.,  Stam, M. (2017). Genome-wide mapping of transcriptional enhancer candidates using DNA and chromatin features in maize. <i>Genome Biology</i>, <b>18</b>(1). https://doi.org/10.1186/S13059-017-1273-4 </div>

8.  <div class="csl-entry">Fagny, M., Kuijjer, M. L., Stam, M., Joets, J., Turc, O., Rozière, J., Pateyron, S., Venon, A., Vitte, C. (2021). Identification of Key Tissue-Specific, Biological Processes by Integrating Enhancer Information in Maize Gene Regulatory Networks. <i>Frontiers in Genetics</i>, <b>11</b>:1703. https://doi.org/10.3389/FGENE.2020.606285/BIBTEX </div>

9.  <div class="csl-entry">Grant, C. E., Bailey, T. L., Noble, W. S.(2011). FIMO: scanning for occurrences of a given motif. <i>Bioinformatics</i>, <b>27</b>(7):1017–1018. https://doi.org/10.1093/bioinformatics/btr064 </div>